In [1]:
%pip install pandas numpy xgboost scikit-learn ta matplotlib

import pandas as pd 
import numpy as np
from xgboost import XGBClassifier, XGBRegressor 
from sklearn.metrics import accuracy_score 
import ta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.7 MB/s eta 0:00:0000:0100:010m
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=05d5a367d5b26857bb26082a9f4d7d1c08974949a448b22cdd1ec07c42579fa6
  Stored in directory: /Users/jamesng/Library/Caches/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_excel('D:/XGBoost/data/ks200.xls')
df = df.rename(columns = {'PRE_END' : 'CLOSE'})
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[['CLOSE']])
df["CLOSE"]= scaled

FileNotFoundError: [Errno 2] No such file or directory: 'D:/XGBoost/data/ks200.xls'

In [ ]:
df1= df[df["DATEDEAL"]>= 20091101]
df1= df1[df1["DATEDEAL"]< 20171230]
df1.reset_index(inplace= True)
df = df1
df

In [ ]:
def Make_Indicators(df):
    global sma, ema, sto_k, sto_d, rsi, macd, disparity
    sma = ta.trend.sma_indicator(df["CLOSE"], 12)
    ema = ta.trend.ema_indicator(df["CLOSE"],12)
    sto_k = ta.momentum.stochrsi_k(df["CLOSE"])
    sto_d = ta.momentum.stochrsi_d(df["CLOSE"])
    rsi = ta.momentum.rsi(df["CLOSE"], 14)
    macd = ta.trend.macd(df["CLOSE"], 13, 26)
    df["MA"]= df["CLOSE"].rolling(20).mean()
    disparity = 100*(df["CLOSE"]/df["MA" ])

In [ ]:
def Concat_ind(dataFrame) :
    global x, y
    
    x = np.zeros((len(df)-40-1,8) )
    idx = 0
    for i in range(40, len(df)-1):
        x[idx][0]= sma[i]
        x[idx][1]= ema[i]
        x[idx][2]= sto_k[i]
        x[idx][3]= sto_d[i]
        x[idx][4]= rsi[i]
        x[idx][5]= macd[i]
        x[idx][6]= disparity[i]
        x[idx][7]= df["'CLOSE"][i]
        idx += 1
        
    y = np.zeros((len(df)-40-1,1))
    idx2 = 0
    for j in range(40, len(df)-1) :
        y[idx2][0]= df["CLOSE"][j+1]
        idx2 += 1

In [ ]:
def Slice_Window(size) :
    global X, Y
    X =[]
    Y =[]
    size = size
    idx = 0
    for i in range(0, len(x)-size+1):
        x_1 =[]
        y_1 =[]
        for j in range(size):
            x_1.append(x[i+j][0])
            x_1.append(x[i+j][1])
            x_1.append(x[i+j][2])
            x_1.append(x[i+j][3])
            x_1.append(x[i+j][4])
            x_1.append(x[i+j][5])
            x_1.append(x[i+j][6])
            x_1.append(x[i+j][7])
            
        y_1.append(y[i+(size-1)][0])
        X.append(x_1)
        Y.append(y_1)
    print(np.shape(X))

In [ ]:
Make_Indicators(df)
Concat_ind(df)
Slice_Window(10)

In [ ]:
model = XGBRegressor(max_depth =5, n_estimators = 300, learning_rate = 0.05)
model.fit(X[:1400], Y[:1400])
y_pred = model.predict(X[1400:])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(y_pred, label = 'pred')
plt.plot(Y[899:], label = 'ori')
plt.legend()

In [ ]:
same_count = 0
Y2 = Y[1399:]
for i in range(len(y_pred)):
  if i> 0:
    cha = y_pred[i] - y_pred[i-1]
    cha2 = Y2 [i] [0] - Y2[i-1][0]
    if cha >= 0 and cha2 >= 0:
      same_count += 1
    if cha < 0 and cha2 < 0:
      same_count += 1

acc = round (same_count / (len (y_pred)-1) *100, 2)
print("Accuracy : {}%".format(acc))